# Using an output handler for validating agent's output

As no one can guarantee the validity or integrity of a raw LLM output, getting a sensible and/or structured output from an agent is tricky. In motleycrew, we addressed this issue with a novel approach: please welcome **output handlers**.

As of now, tool calling is the most reliable way to get a structured output from an LLM. So why not have an output handler - a regular tool with an input schema and a description which instructs the agent to return the output only via this tool. Moreover, we can force the agent into doing this: if it attempts to finish the regular way, we intercept the message and remind the agent that it must call the tool instead.

In [1]:
from dotenv import load_dotenv
from langchain_community.tools import DuckDuckGoSearchRun

from motleycrew import MotleyCrew
from motleycrew.agents.langchain.tool_calling_react import ReActToolCallingAgent
from motleycrew.common import configure_logging
from motleycrew.tasks import SimpleTask

from motleycrew.common.exceptions import InvalidOutput

from langchain_core.tools import StructuredTool

/Users/whimo/motleycrew/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We'll be prompting our agent to create a report on the latest advancements in AI.  
Say we want it to also include some info on AI applications in medicine if it did not already. So our output handler will look for the keyword "medicine" in the agent's output.

Also, we can do any data manipulations we want inside the output handler. In this case, let's just place the output into a certain field.

In [2]:
def check_output(output: str):
    if "medicine" not in output.lower():
        raise InvalidOutput("Add more information about AI applications in medicine.")

    return {"checked_output": output}

output_handler = StructuredTool.from_function(
    name="output_handler",
    description="Output handler",
    func=check_output,
)  # Schema is inferred from the function signature, but you can also specify it explicitly

Now let's create our agent and launch the crew!

In [3]:
search_tool = DuckDuckGoSearchRun()
researcher = ReActToolCallingAgent(
    tools=[search_tool],
    verbose=True,
    chat_history=True,
    output_handler=output_handler,
)

crew = MotleyCrew()
task = SimpleTask(
    crew=crew,
    name="produce comprehensive analysis report on AI advancements in 2024",
    description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.
  Your final answer MUST be a full analysis report""",
    agent=researcher,
)

2024-06-18 18:06:55,525 - motleycrew - WARNING - No known Cypher type matching annotation typing.Optional[typing.Any], will use JSON string
2024-06-18 18:06:55,530 - motleycrew - WARNING - No known Cypher type matching annotation typing.List[str], will use JSON string


In [4]:
crew.run()

2024-06-18 18:07:03,577 - motleycrew - WARNING - No known Cypher type matching annotation typing.List[str], will use JSON string
2024-06-18 18:07:03,727 - motleycrew - WARNING - Lunary public key is not set, tracking will be disabled




> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'latest advancements in AI 2024'}`


In 2024, generative AI might actually become useful for the regular, non-tech person, and we are going to see more people tinkering with a million little AI models. State-of-the-art AI models ... If 2023 was the year the world discovered generative AI (gen AI), 2024 is the year organizations truly began using—and deriving business value from—this new technology.In the latest McKinsey Global Survey on AI, 65 percent of respondents report that their organizations are regularly using gen AI, nearly double the percentage from our previous survey just ten months ago. Now. When OpenAI launched a free web app called ChatGPT in November 2022, nobody knew what was coming. But that low-key release changed everything. By January, ChatGPT had become the fastest ... Here are some important current AI trends to look out for in the coming year. Reality check: more realistic expe

2024-06-18 18:09:29,495 - motleycrew - WARNING - No known Cypher type matching annotation typing.Optional[typing.Any], will use JSON string



Invoking: `output_handler` with `{'output': '**Comprehensive Analysis of AI Advancements in 2024**\n\n**1. Key Trends:**\n- **Generative AI Utilization:** 2024 has seen a significant increase in the practical use of generative AI across various sectors. Organizations are now deriving substantial business value from these technologies, with a notable rise in adoption rates from the previous year.\n- **Realistic Expectations and Multimodal AI:** There is a shift towards more realistic expectations of AI capabilities, coupled with advancements in multimodal AI, which integrates multiple forms of data (e.g., text, image, sound) to enhance learning and decision-making processes.\n- **Open Source AI Models:** The adoption of open source AI models has surged, allowing businesses to integrate these models with private or real-time data, thus enhancing productivity and cost-efficiency.\n\n**2. Breakthrough Technologies:**\n- **Small and Customizable AI Models:** There is a trend towards smalle

[TaskUnit(status=done)]

As you can see in the logs, our medicine requirement was successfully fed back to the agent!

In [6]:
from IPython.display import display, Markdown
display(Markdown(task.output["checked_output"]))

**Comprehensive Analysis of AI Advancements in 2024**

**1. Key Trends:**
- **Generative AI Utilization:** 2024 has seen a significant increase in the practical use of generative AI across various sectors. Organizations are now deriving substantial business value from these technologies, with a notable rise in adoption rates from the previous year.
- **Realistic Expectations and Multimodal AI:** There is a shift towards more realistic expectations of AI capabilities, coupled with advancements in multimodal AI, which integrates multiple forms of data (e.g., text, image, sound) to enhance learning and decision-making processes.
- **Open Source AI Models:** The adoption of open source AI models has surged, allowing businesses to integrate these models with private or real-time data, thus enhancing productivity and cost-efficiency.

**2. Breakthrough Technologies:**
- **Small and Customizable AI Models:** There is a trend towards smaller, more efficient AI models that are easier to train and deploy. These models are increasingly open-source, allowing for greater customization and flexibility in their application.
- **Advancements in Generative AI:** Generative AI continues to be a focal point of technological breakthroughs, with applications expanding into everyday use for non-technical individuals and becoming integral in business operations.

**3. Industry Impacts:**
- **Enhanced Productivity and Innovation:** Generative AI is accelerating innovation cycles, particularly in the technology sector, by enabling the analysis of large data volumes and generating insights at unprecedented speeds.
- **Software Industry Transformation:** The rise of generative AI is reshaping the enterprise software sector, impacting user segments, value pools, and industry dynamics. This shift presents both significant opportunities and challenges for software leaders.
- **Cost Efficiency and Business Transformation:** AI technologies are increasingly embedded into standard business applications, driving cost reductions and automating key processes across industries.

**4. AI in Medicine:**
- **Multimodal Applications in Healthcare:** AI is increasingly going multimodal in medical applications, integrating data from medical images, textual clinical reports, and voice to enhance diagnostics and treatment planning. This integration is expected to drive new understandings in health and healthcare, particularly in areas requiring extensive data analysis.

**Conclusion:**
The landscape of AI in 2024 is marked by significant advancements in generative AI, the rise of open source and customizable models, and a pragmatic approach to AI expectations and applications. These developments are transforming industries by enhancing productivity, fostering innovation, and reshaping business operations. As AI continues to evolve, its integration into daily business practices and decision-making processes is expected to deepen, offering substantial benefits and posing new challenges.